<a href="https://colab.research.google.com/github/pachterlab/CWGFLHGCCHAP_2021/blob/master/notebooks/Preprocessing/cDNAFiltering/kallistoBusRuns_StarvAndStim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!date

In [1]:
#Install kallisto and bustools

!wget --quiet https://github.com/pachterlab/kallisto/releases/download/v0.46.2/kallisto_linux-v0.46.2.tar.gz
!tar -xf kallisto_linux-v0.46.2.tar.gz
!cp kallisto/kallisto /usr/local/bin/

!wget --quiet https://github.com/BUStools/bustools/releases/download/v0.40.0/bustools_linux-v0.40.0.tar.gz
!tar -xf bustools_linux-v0.40.0.tar.gz
!cp bustools/bustools /usr/local/bin/

In [13]:
import requests
from tqdm import tnrange, tqdm_notebook
def download_file(doi,ext):
    url = 'https://api.datacite.org/dois/'+doi+'/media'
    r = requests.get(url).json()
    netcdf_url = r['data'][0]['attributes']['url']
    r = requests.get(netcdf_url,stream=True)
    #Set file name
    fname = doi.split('/')[-1]+ext
    #Download file with progress bar
    if r.status_code == 403:
        print("File Unavailable")
    if 'content-length' not in r.headers:
        print("Did not get file")
    else:
        with open(fname, 'wb') as f:
            total_length = int(r.headers.get('content-length'))
            pbar = tnrange(int(total_length/1024), unit="B")
            for chunk in r.iter_content(chunk_size=1024):
                if chunk:
                    pbar.update()
                    f.write(chunk)
        return fname

In [ ]:
#Get reference data () fastq's)

#Transcripts
#!wget --quiet https://caltech.box.com/shared/static/use8yd40nlif8147y2vaf0j4j3wjy3vm
download_file('10.22002/D1.1825','.gz')

#Gff3 (TRINITY)
#!wget --quiet https://caltech.box.com/shared/static/bh6ivy8xpatkwtw61cj50ahkt93kzuwv
download_file('10.22002/D1.1824','.gz')



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`


'D1.1824.gz'

In [ ]:
# Get doi links for all Starvation cDNA fastq.gz files
starvFiles = []
dois = [] #16 numbers
for doi in dois:
  url = 'https://api.datacite.org/dois/'+doi+'/media'
  r = requests.get(url).json()
  netcdf_url = r['data'][0]['attributes']['url']

  starvFiles += [netcdf_url]

s1 = starvFiles[0]
s2 = starvFiles[1]
s3 = starvFiles[2]
s4 = starvFiles[3]
s5 = starvFiles[4]
s6 = starvFiles[5]
s7 = starvFiles[6]
s8 = starvFiles[7]

s9 = starvFiles[8]
s10 = starvFiles[9]
s11 = starvFiles[10]
s12 = starvFiles[11]
s13 = starvFiles[12]
s14 = starvFiles[13]
s15 = starvFiles[14]
s16 = starvFiles[15]

In [ ]:
# Get doi links for all Stimulation cDNA fastq.gz files
stimFiles = []
dois = [] #8 numbers
for doi in dois:
  url = 'https://api.datacite.org/dois/'+doi+'/media'
  r = requests.get(url).json()
  netcdf_url = r['data'][0]['attributes']['url']

  stimFiles += [netcdf_url]


stim1 = stimFiles[0]
stim2 = stimFiles[1]
stim3 = stimFiles[2]
stim4 = stimFiles[3]
stim5 = stimFiles[4]
stim6 = stimFiles[5]
stim7 = stimFiles[6]
stim8 = stimFiles[7]


In [ ]:
#Get original, CellRanger data
#!wget --quiet https://caltech.box.com/shared/static/ts2i8pid7uky28kke4duyl8jwugx8z4h
download_file('10.22002/D1.1798','.gz')

#Names from Stim ClickTags
#!wget --quiet https://caltech.box.com/shared/static/saiqluf1i7m8jx6vwaf7b4z2psq7cv27

download_file('10.22002/D1.1817','.gz')



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`


In [ ]:
!gunzip *.gz

In [16]:
!pip install --quiet anndata
!pip install --quiet scanpy
!pip install --quiet louvain

     |████████████████████████████████| 122kB 11.4MB/s 
     |████████████████████████████████| 7.7MB 12.5MB/s 
     |████████████████████████████████| 71kB 8.4MB/s 
     |████████████████████████████████| 51kB 6.0MB/s 
     |████████████████████████████████| 2.2MB 11.8MB/s 
     |████████████████████████████████| 3.2MB 51.6MB/s 


In [17]:
import pandas as pd
import anndata
import scanpy as sc
import numpy as np
import scipy.sparse

import matplotlib.pyplot as plt
%matplotlib inline
sc.set_figure_params(dpi=125)

### **Run kallisto bus on data with Starvation data**



In [ ]:
#Make Kallisto index (referene https://www.kallistobus.tools/getting_started)
!mv D1.1825 transcripts.fa
!kallisto index -i clytia_trin.idx -k 31 transcripts.fa


[build] loading fasta file transcripts.fa
[build] k-mer length: 31
[build] warning: clipped off poly-A tail (longer than 10)
        from 30 target sequences
[build] warning: replaced 205989 non-ACGUT characters in the input sequence
        with pseudorandom nucleotides
[build] counting k-mers ... tcmalloc: large alloc 1610612736 bytes == 0x68af6000 @  0x7f8f107161e7 0x6f46bd 0x6f4739 0x4af899 0x4a7a68 0x4aec09 0x44e175 0x7f8f0f732bf7 0x452f19
done.
[build] building target de Bruijn graph ...  done 
[build] creating equivalence classes ...  done
[build] target de Bruijn graph has 410850 contigs and contains 61068880 k-mers 



Run kallisto for one set of samples

In [ ]:
#Create BUS files from fastq's, can't do separate lines
!mkfifo R1.gz R2.gz R1_02.gz R2_02.gz R1_03.gz R2_03.gz R1_04.gz R2_04.gz; curl -Ls $s1 > R1.gz & curl -Ls $s2 > R2.gz & curl -Ls $s3 > R1_02.gz & curl -Ls $s4 > R2_02.gz & curl -Ls $s5 > R1_03.gz & curl -Ls $s6 > R2_03.gz & curl -Ls $s7 > R1_04.gz & curl -Ls $s8 > R2_04.gz &
kallisto bus -i clytia_trin.idx -o bus_output/ -x 10xv2 -t 8 \
/home/tchari/cDNA_fastqs/FT-BB01610/FT-SA16888_FT-TS91279/FT-SA16888_S1_L004_R1_001.fastq.gz \
/home/tchari/cDNA_fastqs/FT-BB01610/FT-SA16888_FT-TS91279/FT-SA16888_S1_L004_R2_001.fastq.gz \
/home/tchari/cDNA_fastqs/FT-BB01610/FT-SA16888_FT-TS91279/FT-SA16889_S2_L004_R1_001.fastq.gz \
/home/tchari/cDNA_fastqs/FT-BB01610/FT-SA16888_FT-TS91279/FT-SA16889_S2_L004_R2_001.fastq.gz \
/home/tchari/cDNA_fastqs/FT-BB01610/FT-SA16888_FT-TS91279/FT-SA16890_S3_L004_R1_001.fastq.gz \
/home/tchari/cDNA_fastqs/FT-BB01610/FT-SA16888_FT-TS91279/FT-SA16890_S3_L004_R2_001.fastq.gz \
/home/tchari/cDNA_fastqs/FT-BB01610/FT-SA16888_FT-TS91279/FT-SA16891_S4_L004_R1_001.fastq.gz \
/home/tchari/cDNA_fastqs/FT-BB01610/FT-SA16888_FT-TS91279/FT-SA16891_S4_L004_R2_001.fastq.gz


In [ ]:
#Generate gene-count matrices
!wget --quiet https://github.com/bustools/getting_started/releases/download/getting_started/10xv2_whitelist.txt

#Make t2g file
!mv D1.1824 trinity.gff3
!awk '{ print $12"\t"$10}' trinity.gff3  > t2g_rough.txt
!sed 's/[";]//g' t2g_rough.txt > t2g_trin.txt




In [ ]:
#!cd bus_output/
!mkdir bus_output/genecount/ bus_output/tmp/

!bustools correct -w 10xv2_whitelist.txt -p bus_output/output.bus | bustools sort -T bus_output/tmptmp/ -t 8 -p - | bustools count -o bus_output/genecount/genes -g t2g_trin.txt -e bus_output/matrix.ec -t bus_output/transcripts.txt --genecounts -

In [ ]:
!ls test

Run kallisto for other sample set

In [ ]:
#Create BUS files from fastq's
!mkfifo R1.gz R2.gz R1_02.gz R2_02.gz R1_03.gz R2_03.gz R1_04.gz R2_04.gz; curl -Ls $s9 > R1.gz & curl -Ls $s10 > R2.gz & curl -Ls $s11 > R1_02.gz & curl -Ls $s12 > R2_02.gz & curl -Ls $s13 > R1_03.gz & curl -Ls $s14 > R2_03.gz & curl -Ls $s15 > R1_04.gz & curl -Ls $s16 > R2_04.gz &
!kallisto bus -i clytia_trin.idx -o bus_output_02/ -x 10xv2 -t 8 \
/home/tchari/cDNA_fastqs/FT-BB01610/FT-SA16892_FT-TS91283/FT-SA16892_S5_L004_R1_001.fastq.gz \
/home/tchari/cDNA_fastqs/FT-BB01610/FT-SA16892_FT-TS91283/FT-SA16892_S5_L004_R2_001.fastq.gz \
/home/tchari/cDNA_fastqs/FT-BB01610/FT-SA16892_FT-TS91283/FT-SA16893_S6_L004_R1_001.fastq.gz \
/home/tchari/cDNA_fastqs/FT-BB01610/FT-SA16892_FT-TS91283/FT-SA16893_S6_L004_R2_001.fastq.gz \
/home/tchari/cDNA_fastqs/FT-BB01610/FT-SA16892_FT-TS91283/FT-SA16894_S7_L004_R1_001.fastq.gz \
/home/tchari/cDNA_fastqs/FT-BB01610/FT-SA16892_FT-TS91283/FT-SA16894_S7_L004_R2_001.fastq.gz \
/home/tchari/cDNA_fastqs/FT-BB01610/FT-SA16892_FT-TS91283/FT-SA16895_S8_L004_R1_001.fastq.gz \
/home/tchari/cDNA_fastqs/FT-BB01610/FT-SA16892_FT-TS91283/FT-SA16895_S8_L004_R2_001.fastq.gz


In [ ]:
#Generate gene-count matrices
!cd bus_output_02/
!mkdir bus_output_02/genecount/ bus_output_02/tmp/

!bustools correct -w 10xv2_whitelist.txt -p bus_output_02/output.bus | bustools sort -T bus_output_02/tmp/ -t 8 -p -  | bustools count -o bus_output_02/genecount/genes -g t2g_trin.txt -e bus_output_02/matrix.ec -t bus_output_02/transcripts.txt --genecounts -

Merge matrices (Add -1 to first and -2 to second dataset)

In [ ]:
path = "bus_output/genecount/"
jelly_adata_01 = sc.read(path+'genes.mtx', cache=True)
jelly_adata_01.var_names = pd.read_csv(path+'genes.genes.txt', header=None)[0]
jelly_adata_01.obs_names = pd.read_csv(path+'genes.barcodes.txt', header=None)[0]

jelly_adata_01.obs_names = [i+"-1" for i in jelly_adata_01.obs_names]

In [ ]:
path = "bus_output_02/genecount/"
jelly_adata_02 = sc.read(path+'genes.mtx', cache=True)
jelly_adata_02.var_names = pd.read_csv(path+'genes.genes.txt', header=None)[0]
jelly_adata_02.obs_names = pd.read_csv(path+'genes.barcodes.txt', header=None)[0]

jelly_adata_02.obs_names = [i+"-2" for i in jelly_adata_02.obs_names]

In [ ]:
jelly_adata = jelly_adata_01.concatenate(jelly_adata_02,join='outer', index_unique=None)

In [ ]:
# Filter barcodes by 'real' cells
cellR = anndata.read('D1.1798')
cells = list(cellR.obs_names)

jelly_adata = jelly_adata[cells,:]

In [ ]:
jelly_adata.write('fedStarved_raw.h5ad')

### **Run kallisto bus on data with Stimulation data**




Run kallisto for one set of samples

In [ ]:
#Create BUS files from fastq's, can't do separate lines
!kallisto bus -i clytia_trin.idx -o bus_output/ -x 10xv3 -t 8 \
/home/tchari/cDNA_fastqs/FT-BB01610/FT-SA16888_FT-TS91279/FT-SA16888_S1_L004_R1_001.fastq.gz \
/home/tchari/cDNA_fastqs/FT-BB01610/FT-SA16888_FT-TS91279/FT-SA16888_S1_L004_R2_001.fastq.gz \
/home/tchari/cDNA_fastqs/FT-BB01610/FT-SA16888_FT-TS91279/FT-SA16889_S2_L004_R1_001.fastq.gz \
/home/tchari/cDNA_fastqs/FT-BB01610/FT-SA16888_FT-TS91279/FT-SA16889_S2_L004_R2_001.fastq.gz \
/home/tchari/cDNA_fastqs/FT-BB01610/FT-SA16888_FT-TS91279/FT-SA16890_S3_L004_R1_001.fastq.gz \
/home/tchari/cDNA_fastqs/FT-BB01610/FT-SA16888_FT-TS91279/FT-SA16890_S3_L004_R2_001.fastq.gz \
/home/tchari/cDNA_fastqs/FT-BB01610/FT-SA16888_FT-TS91279/FT-SA16891_S4_L004_R1_001.fastq.gz \
/home/tchari/cDNA_fastqs/FT-BB01610/FT-SA16888_FT-TS91279/FT-SA16891_S4_L004_R2_001.fastq.gz


In [ ]:
#Generate gene-count matrices
!wget --quiet https://github.com/bustools/getting_started/releases/download/getting_started/10xv3_whitelist.txt


In [ ]:
#!cd bus_output/
!mkdir bus_output/genecount/ bus_output/tmp/

!bustools correct -w 10xv3_whitelist.txt -p bus_output/output.bus | bustools sort -T bus_output/tmptmp/ -t 8 -p - | bustools count -o bus_output/genecount/genes -g t2g_trin.txt -e bus_output/matrix.ec -t bus_output/transcripts.txt --genecounts -

In [ ]:
!ls test

Run kallisto for other sample set

In [ ]:
#Create BUS files from fastq's

!kallisto bus -i clytia_trin.idx -o bus_output_02/ -x 10xv3 -t 8 \
/home/tchari/cDNA_fastqs/FT-BB01610/FT-SA16892_FT-TS91283/FT-SA16892_S5_L004_R1_001.fastq.gz \
/home/tchari/cDNA_fastqs/FT-BB01610/FT-SA16892_FT-TS91283/FT-SA16892_S5_L004_R2_001.fastq.gz \
/home/tchari/cDNA_fastqs/FT-BB01610/FT-SA16892_FT-TS91283/FT-SA16893_S6_L004_R1_001.fastq.gz \
/home/tchari/cDNA_fastqs/FT-BB01610/FT-SA16892_FT-TS91283/FT-SA16893_S6_L004_R2_001.fastq.gz \
/home/tchari/cDNA_fastqs/FT-BB01610/FT-SA16892_FT-TS91283/FT-SA16894_S7_L004_R1_001.fastq.gz \
/home/tchari/cDNA_fastqs/FT-BB01610/FT-SA16892_FT-TS91283/FT-SA16894_S7_L004_R2_001.fastq.gz \
/home/tchari/cDNA_fastqs/FT-BB01610/FT-SA16892_FT-TS91283/FT-SA16895_S8_L004_R1_001.fastq.gz \
/home/tchari/cDNA_fastqs/FT-BB01610/FT-SA16892_FT-TS91283/FT-SA16895_S8_L004_R2_001.fastq.gz


In [ ]:
#Generate gene-count matrices
!cd bus_output_02/
!mkdir bus_output_02/genecount/ bus_output_02/tmp/

!bustools correct -w 10xv3_whitelist.txt -p bus_output_02/output.bus | bustools sort -T bus_output_02/tmp/ -t 8 -p -  | bustools count -o bus_output_02/genecount/genes -g t2g_trin.txt -e bus_output_02/matrix.ec -t bus_output_02/transcripts.txt --genecounts -

Merge matrices (Add -1 to first and -2 to second dataset)

In [ ]:
path = "bus_output/genecount/"
jelly_adata_01 = sc.read(path+'genes.mtx', cache=True)
jelly_adata_01.var_names = pd.read_csv(path+'genes.genes.txt', header=None)[0]
jelly_adata_01.obs_names = pd.read_csv(path+'genes.barcodes.txt', header=None)[0]

jelly_adata_01.obs_names = [i+"-1" for i in jelly_adata_01.obs_names]

In [ ]:
path = "bus_output_02/genecount/"
jelly_adata_02 = sc.read(path+'genes.mtx', cache=True)
jelly_adata_02.var_names = pd.read_csv(path+'genes.genes.txt', header=None)[0]
jelly_adata_02.obs_names = pd.read_csv(path+'genes.barcodes.txt', header=None)[0]

jelly_adata_02.obs_names = [i+"-2" for i in jelly_adata_02.obs_names]

In [ ]:
jelly_adata = jelly_adata_01.concatenate(jelly_adata_02,join='outer', index_unique=None)

In [ ]:
# Filter barcodes by 'real' cells
!mv D1.1817 jelly4stim_individs_tagCells_50k.mat
barcodes_list = sio.loadmat('jelly4stim_individs_tagCells_50k.mat')
barcodes_list.pop('__header__', None)
barcodes_list.pop('__version__', None)
barcodes_list.pop('__globals__', None)

# Add all cell barcodes for each individual
barcodes = []
for b in barcodes_list:
    if barcodes_list[b] != "None":
        barcodes.append(b)

print(len(barcodes))

barcodes = [s.replace('-1', '-3') for s in barcodes]
barcodes = [s.replace('-2', '-1') for s in barcodes]
barcodes = [s.replace('-3', '-2') for s in barcodes]


In [ ]:
jelly_adata = jelly_adata[barcodes,:]

In [ ]:
jelly_adata.write('stimulation_raw.h5ad')